In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
import os
import random
import shutil
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import random
import shutil
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import train_test_split



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Step 1: Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define the path to the directory containing ship and no-ship folders
EXTRACT_PATH = "/content/drive/MyDrive/archive"   # Change if directory is elsewhere

# Class folders
SHIP_DIR = os.path.join(EXTRACT_PATH, "ship")
NO_SHIP_DIR = os.path.join(EXTRACT_PATH, "no-ship")

# Step 3: Collect all images with their labels
all_images = []
for img in os.listdir(SHIP_DIR):
    all_images.append((os.path.join(SHIP_DIR, img), "ship"))
for img in os.listdir(NO_SHIP_DIR):
    all_images.append((os.path.join(NO_SHIP_DIR, img), "no-ship"))

# Step 4: Create DataFrame from images
df = pd.DataFrame(all_images, columns=["filepath", "label"])

# Step 5: Split into Train / Val / Test
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

# Step 6: Create directories for YOLO
for split in ["train", "val", "test"]:
    for class_name in ["ship", "no-ship"]:
        os.makedirs(f"/content/yolo_data/{split}/{class_name}", exist_ok=True)

# Step 7: Copy files according to the split
def process_split(df, split):
    for _, row in df.iterrows():
        class_name = row["label"]
        dest_dir = os.path.join("/content/yolo_data", split, class_name)
        shutil.copy(row["filepath"], os.path.join(dest_dir, os.path.basename(row["filepath"])))

process_split(train_df, "train")
process_split(val_df, "val")
process_split(test_df, "test")

Mounted at /content/drive


In [10]:
import re

# Step 8: Save filenames from each set (based on enhanced images)
train_names = train_df["filepath"].apply(os.path.basename).tolist()
val_names = val_df["filepath"].apply(os.path.basename).tolist()
test_names = test_df["filepath"].apply(os.path.basename).tolist()

# Step 9: Extract ZIP file of original images (if not already done)
import zipfile

zip_path = "/content/archive2.zip"  # ← Update if different
extract_path = "/content/original_data/archive2/archive2/archive2/archive2/"

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# After extraction - expecting structure: /content/original_data/ship + no-ship
ORIG_SHIP_DIR = os.path.join(extract_path, "ship")
ORIG_NO_SHIP_DIR = os.path.join(extract_path, "no-ship")

# Step 10: Create directories for yolo_data_original
base_output = "/content/yolo_data_original2"

for split in ["train", "val", "test"]:
    for cls in ["ship", "no-ship"]:
        os.makedirs(os.path.join(base_output, split, cls), exist_ok=True)

# Step 11: Helper to decode original filenames from enhanced names
def extract_index(filename):
    match = re.search(r'_(\d{6})_', filename)
    return match.group(1) if match else None

def build_original_filename(enhanced_name):
    # Extract category from name
    if enhanced_name.startswith("ship"):
        prefix = "ship"
    elif enhanced_name.startswith("no-ship"):
        prefix = "no-ship"
    else:
        return None, None  # No category identified

    # Extract index
    index = extract_index(enhanced_name)
    if index:
        filename = f"{prefix}_{index}_original.png"
        return filename, prefix
    else:
        return None, None

# Step 12: Copy original images according to names
def copy_matching_images(enhanced_file_list, split_name):
    for enhanced_name in enhanced_file_list:
        orig_name, cls = build_original_filename(enhanced_name)
        if orig_name and cls:
            src_dir = ORIG_SHIP_DIR if cls == "ship" else ORIG_NO_SHIP_DIR
            dest_dir = os.path.join(base_output, split_name, cls)

            src_path = os.path.join(src_dir, orig_name)
            if os.path.exists(src_path):
                shutil.copy(src_path, os.path.join(dest_dir, orig_name))

copy_matching_images(train_names, "train")
copy_matching_images(val_names, "val")
copy_matching_images(test_names, "test")

In [11]:

# Number of folds to use
NUM_FOLDS = 10
# Number of training epochs
NUM_EPOCHS = 10
# Size of the separate test set (10%)
TEST_SIZE = 0.1

def prepare_data_for_kfold(enhanced_dir, original_dir):
    """Prepare data for k-fold validation"""

    # Collect images from all sets (train, val, test) into one place
    enhanced_ship_images = []
    enhanced_noship_images = []
    original_ship_images = []
    original_noship_images = []

    # For enhanced data
    for split in ['train', 'val', 'test']:
        ship_path = os.path.join(enhanced_dir, split, 'ship')
        noship_path = os.path.join(enhanced_dir, split, 'no-ship')

        if os.path.exists(ship_path):
            for img in os.listdir(ship_path):
                enhanced_ship_images.append(os.path.join(ship_path, img))

        if os.path.exists(noship_path):
            for img in os.listdir(noship_path):
                enhanced_noship_images.append(os.path.join(noship_path, img))

    # For original data
    for split in ['train', 'val', 'test']:
        ship_path = os.path.join(original_dir, split, 'ship')
        noship_path = os.path.join(original_dir, split, 'no-ship')

        if os.path.exists(ship_path):
            for img in os.listdir(ship_path):
                original_ship_images.append(os.path.join(ship_path, img))

        if os.path.exists(noship_path):
            for img in os.listdir(noship_path):
                original_noship_images.append(os.path.join(noship_path, img))

    # Create DataFrame with paths and labels
    enhanced_data = pd.DataFrame({
        'filepath': enhanced_ship_images + enhanced_noship_images,
        'label': ['ship'] * len(enhanced_ship_images) + ['no-ship'] * len(enhanced_noship_images)
    })

    original_data = pd.DataFrame({
        'filepath': original_ship_images + original_noship_images,
        'label': ['ship'] * len(original_ship_images) + ['no-ship'] * len(original_noship_images)
    })

    return enhanced_data, original_data

def split_train_test(data_df, test_size=TEST_SIZE, random_state=42):
    """Split data into training and test sets"""

    # Split while preserving class distribution
    train_df, test_df = train_test_split(
        data_df,
        test_size=test_size,
        random_state=random_state,
        stratify=data_df['label']  # Maintains the same ship/no-ship ratio in both sets
    )

    print(f"  Train set: {len(train_df)} images")
    print(f"  Test set: {len(test_df)} images")

    return train_df, test_df

def perform_kfold_validation(train_df, data_type='enhanced', img_size=320):
    """Perform K-fold validation on the training set"""

    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

    # Store results for later analysis
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
        print(f"\n=== Fold {fold+1}/{NUM_FOLDS} for {data_type} data ===")

        start_time = time.time()

        # Create directories for current fold
        fold_dir = f"/content/kfold_{data_type}/fold_{fold+1}"

        # Remove directory if it exists
        if os.path.exists(fold_dir):
            shutil.rmtree(fold_dir)

        for class_name in ['ship', 'no-ship']:
            for split in ['train', 'val']:
                os.makedirs(f"{fold_dir}/{split}/{class_name}", exist_ok=True)

        # Split into train and validation
        fold_train_df = train_df.iloc[train_idx].reset_index(drop=True)
        fold_val_df = train_df.iloc[val_idx].reset_index(drop=True)

        print(f"  Fold {fold+1} - Train: {len(fold_train_df)}, Validation: {len(fold_val_df)}")

        # Copy files to appropriate directories
        for _, row in fold_train_df.iterrows():
            dest_path = f"{fold_dir}/train/{row['label']}/{os.path.basename(row['filepath'])}"
            shutil.copy(row['filepath'], dest_path)

        for _, row in fold_val_df.iterrows():
            dest_path = f"{fold_dir}/val/{row['label']}/{os.path.basename(row['filepath'])}"
            shutil.copy(row['filepath'], dest_path)

        # Train the model
        model = YOLO("yolov8n-cls.pt")

        # Set training parameters
        results = model.train(
            data=fold_dir,
            epochs=NUM_EPOCHS,
            imgsz=img_size,
            verbose=False,
            project=f"/content/runs/kfold_{data_type}",
            name=f"fold_{fold+1}"
        )

        # Evaluate the model
        metrics = model.val()

        # Save results
        fold_results.append({
            'fold': fold + 1,
            'accuracy': metrics.top1,
            'val_indices': val_idx,  # Store indices for unified predictions
            'model_path': f"/content/runs/kfold_{data_type}/fold_{fold+1}/weights/best.pt"
        })

        end_time = time.time()
        fold_time = end_time - start_time

        print(f"Fold {fold+1} completed in {fold_time:.2f} seconds")
        print(f"Fold {fold+1} accuracy: {metrics.top1:.4f}")

    # Calculate statistics
    accuracies = [result['accuracy'] for result in fold_results]
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    print(f"\n=== {data_type.upper()} DATA K-FOLD RESULTS ===")
    print(f"Mean accuracy: {mean_accuracy:.4f}")
    print(f"Standard deviation: {std_accuracy:.4f}")
    print(f"Accuracy per fold: {accuracies}")

    return fold_results, mean_accuracy, std_accuracy

def evaluate_model_on_test_set(best_model_path, test_df, class_names, data_type):
    """Evaluate the best model on the separate test set"""

    print(f"\nEvaluating best model for {data_type} data on the test set...")

    # Load the best model
    model = YOLO(best_model_path)

    # Predict on test set
    predictions = []
    true_labels = []
    confidences = []

    for _, row in test_df.iterrows():
        result = model.predict(row['filepath'], verbose=False)[0]
        pred_class = result.names[result.probs.top1]

        predictions.append(pred_class)
        true_labels.append(row['label'])
        confidences.append(float(result.probs.top1conf))

    # Analyze results
    accuracy = accuracy_score(true_labels, predictions)
    class_report = classification_report(true_labels, predictions, target_names=class_names)
    cm = confusion_matrix(true_labels, predictions, labels=class_names)

    print(f"Test Set Accuracy: {accuracy:.4f}")
    print(f"\nClassification Report:\n{class_report}")

    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {data_type.capitalize()} Data (Test Set)')
    plt.savefig(f'/content/{data_type}_confusion_matrix.png')
    plt.close()

    # Analyze misclassified samples
    error_df = pd.DataFrame({
        'filepath': test_df['filepath'].values,
        'true_label': true_labels,
        'predicted': predictions,
        'confidence': confidences
    })

    error_df['correct'] = error_df['true_label'] == error_df['predicted']
    misclassified = error_df[~error_df['correct']].sort_values('confidence', ascending=False)

    print(f"Total test samples: {len(test_df)}")
    print(f"Correct predictions: {len(error_df[error_df['correct']])}")
    print(f"Misclassified samples: {len(misclassified)}")

    # Save list of misclassified images
    if len(misclassified) > 0:
        misclassified.to_csv(f'/content/{data_type}_misclassified.csv', index=False)
        print(f"Saved misclassified samples to: /content/{data_type}_misclassified.csv")

    return accuracy, class_report, cm, misclassified

def find_best_model(fold_results):
    """Find the best model from all folds"""
    return max(fold_results, key=lambda x: x['accuracy'])

def compare_models(enhanced_results, original_results, enhanced_test_acc, original_test_acc):
    """Compare the models"""

    enhanced_cv_acc = enhanced_results[1]  # Mean accuracy from K-fold
    original_cv_acc = original_results[1]  # Mean accuracy from K-fold
    enhanced_cv_std = enhanced_results[2]  # Standard deviation from K-fold
    original_cv_std = original_results[2]  # Standard deviation from K-fold

    print("\n=== MODEL COMPARISON ===")
    print(f"Enhanced Resolution Model:")
    print(f"  - Cross-validation Accuracy: {enhanced_cv_acc:.4f} (±{enhanced_cv_std:.4f})")
    print(f"  - Test Set Accuracy: {enhanced_test_acc:.4f}")
    print(f"Original Resolution Model:")
    print(f"  - Cross-validation Accuracy: {original_cv_acc:.4f} (±{original_cv_std:.4f})")
    print(f"  - Test Set Accuracy: {original_test_acc:.4f}")

    # Visualization of comparison
    plt.figure(figsize=(12, 6))

    # Data for comparison
    models = ['Enhanced - CV', 'Enhanced - Test', 'Original - CV', 'Original - Test']
    accuracies = [enhanced_cv_acc, enhanced_test_acc, original_cv_acc, original_test_acc]
    errors = [enhanced_cv_std, 0, original_cv_std, 0]  # No std dev for test set
    colors = ['#3498db', '#2980b9', '#2ecc71', '#27ae60']

    # Create chart
    bars = plt.bar(models, accuracies, yerr=errors, capsize=10, color=colors)

    # Add labels
    plt.ylabel('Accuracy')
    plt.title('Model Comparison: Enhanced vs. Original Resolution')
    plt.ylim(max(0.9, min(accuracies) - 0.05), min(1.0, max(accuracies) + 0.05))  # Adjust Y-axis limits

    # Add values above bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{height:.4f}', ha='center', va='bottom')

    plt.savefig('/content/model_comparison.png')
    plt.close()

    # Statistical comparison - is the difference significant
    if enhanced_test_acc > original_test_acc:
        diff_pct = ((enhanced_test_acc - original_test_acc) / original_test_acc) * 100
        print(f"\nEnhanced resolution model performs {diff_pct:.2f}% better on the test set.")
    elif original_test_acc > enhanced_test_acc:
        diff_pct = ((original_test_acc - enhanced_test_acc) / enhanced_test_acc) * 100
        print(f"\nOriginal resolution model performs {diff_pct:.2f}% better on the test set.")
    else:
        print("\nBoth models perform equally on the test set.")

# Main function for evaluation
def main():
    # Measure runtime
    total_start_time = time.time()

    # Define paths
    enhanced_data_dir = "/content/yolo_data"
    original_data_dir = "/content/yolo_data_original2"

    # Define class names
    class_names = ['no-ship', 'ship']

    print("Preparing data for K-fold validation...")
    enhanced_data, original_data = prepare_data_for_kfold(enhanced_data_dir, original_data_dir)

    print(f"Enhanced data: {len(enhanced_data)} images")
    print(f"Original data: {len(original_data)} images")

    # 1. Split into training and test sets
    print("\nSplitting enhanced data into train and test sets...")
    enhanced_train_df, enhanced_test_df = split_train_test(enhanced_data)

    print("\nSplitting original data into train and test sets...")
    original_train_df, original_test_df = split_train_test(original_data)

    # 2. Perform K-fold validation on training set
    print("\nPerforming K-fold validation for enhanced resolution data...")
    enhanced_results = perform_kfold_validation(enhanced_train_df, data_type='enhanced', img_size=320)

    print("\nPerforming K-fold validation for original resolution data...")
    original_results = perform_kfold_validation(original_train_df, data_type='original', img_size=80)

    # 3. Find the best model
    best_enhanced_model = find_best_model(enhanced_results[0])
    best_original_model = find_best_model(original_results[0])

    print(f"\nBest enhanced model: Fold {best_enhanced_model['fold']} with accuracy {best_enhanced_model['accuracy']:.4f}")
    print(f"Best original model: Fold {best_original_model['fold']} with accuracy {best_original_model['accuracy']:.4f}")

    # 4. Evaluate best models on separate test set
    enhanced_test_results = evaluate_model_on_test_set(
        best_enhanced_model['model_path'],
        enhanced_test_df,
        class_names,
        'enhanced'
    )

    original_test_results = evaluate_model_on_test_set(
        best_original_model['model_path'],
        original_test_df,
        class_names,
        'original'
    )

    # 5. Compare models
    compare_models(
        enhanced_results,
        original_results,
        enhanced_test_results[0],  # Accuracy on test set for enhanced model
        original_test_results[0]   # Accuracy on test set for original model
    )

    # Summarize runtime
    total_end_time = time.time()
    total_time = total_end_time - total_start_time

    print(f"\nComplete evaluation process finished in {total_time/60:.2f} minutes")
    print("\nK-fold validation complete! Check the output files for detailed results.")

if __name__ == "__main__":
    main()

Preparing data for K-fold validation...
Enhanced data: 3860 images
Original data: 3858 images

Splitting enhanced data into train and test sets...
  Train set: 3474 images
  Test set: 386 images

Splitting original data into train and test sets...
  Train set: 3472 images
  Test set: 386 images

Performing K-fold validation for enhanced resolution data...

=== Fold 1/10 for enhanced data ===
  Fold 1 - Train: 3126, Validation: 348


100%|██████████| 5.31M/5.31M [00:00<00:00, 394MB/s]


Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/kfold_enhanced/fold_1, epochs=10, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/runs/kfold_enhanced, name=fold_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 5.35M/5.35M [00:00<00:00, 353MB/s]


AMP: checks passed ✅


...


10 epochs completed in 0.016 hours.
Optimizer stripped from /content/runs/kfold_original/fold_10/weights/last.pt, 3.0MB
Optimizer stripped from /content/runs/kfold_original/fold_10/weights/best.pt, 3.0MB

Validating /content/runs/kfold_original/fold_10/weights/best.pt...
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/kfold_original/fold_10/train... found 3125 images in 2 classes ✅ 
val: /content/kfold_original/fold_10/val... found 347 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 11/11 [00:00<00:00, 72.92it/s]


                   all      0.997          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/kfold_original/fold_10
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/kfold_original/fold_10/train... found 3125 images in 2 classes ✅ 
val: /content/kfold_original/fold_10/val... found 347 images in 2 classes ✅ 
test: None...


val: Scanning /content/kfold_original/fold_10/val... 347 images, 0 corrupt: 100%|██████████| 347/347 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 22/22 [00:00<00:00, 79.02it/s]


                   all      0.997          1
Speed: 0.0ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/kfold_original/fold_102
Fold 10 completed in 64.67 seconds
Fold 10 accuracy: 0.9971

=== ORIGINAL DATA K-FOLD RESULTS ===
Mean accuracy: 0.9974
Standard deviation: 0.0024
Accuracy per fold: [0.9942528605461121, 0.9942528605461121, 0.9942362904548645, 1.0, 1.0, 0.9971181750297546, 1.0, 1.0, 0.9971181750297546, 0.9971181750297546]

Best enhanced model: Fold 2 with accuracy 0.9971
Best original model: Fold 4 with accuracy 1.0000

Evaluating best model for enhanced data on the test set...
Test Set Accuracy: 0.9948

Classification Report:
              precision    recall  f1-score   support

     no-ship       1.00      0.99      1.00       286
        ship       0.98      1.00      0.99       100

    accuracy                           0.99       386
   macro avg       0.99      1.00      0.99       386
weighted avg       0.99      0

In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

def extract_metrics_from_csv(data_type, num_folds=10):
    """
    Extract training and validation metrics from the results CSVs
    generated by YOLOv8 during training.

    Args:
        data_type: 'enhanced' or 'original'
        num_folds: Number of folds used in cross-validation

    Returns:
        Dictionary with metrics for each fold
    """
    all_fold_metrics = {}

    for fold in range(1, num_folds + 1):
        csv_path = f"/content/runs/kfold_{data_type}/fold_{fold}/results.csv"

        if not os.path.exists(csv_path):
            print(f"Warning: Results CSV not found for {data_type} data, fold {fold}")
            continue

        # Read results CSV
        metrics_df = pd.read_csv(csv_path)

        # Store in dictionary
        all_fold_metrics[fold] = metrics_df

    return all_fold_metrics

def plot_training_validation_curves(metrics_dict, data_type, metric_name="top1"):
    """
    Plot training vs validation curves for a specific metric across all folds.

    Args:
        metrics_dict: Dictionary with metrics dataframes for each fold
        data_type: 'enhanced' or 'original'
        metric_name: Name of the metric to plot (default is top1 for accuracy)

    Returns:
        None, saves the plot to disk
    """
    plt.figure(figsize=(14, 10))

    # For storing mean values across folds
    mean_train = None
    mean_val = None
    max_epochs = 0

    # Plot each fold
    for fold, df in metrics_dict.items():
        if not f'train/{metric_name}' in df.columns or not f'val/{metric_name}' in df.columns:
            print(f"Warning: Metric {metric_name} not found in fold {fold}")
            continue

        epochs = df['epoch'].values
        train_values = df[f'train/{metric_name}'].values
        val_values = df[f'val/{metric_name}'].values

        # Update max epochs
        max_epochs = max(max_epochs, len(epochs))

        # Initialize mean arrays if needed
        if mean_train is None:
            mean_train = np.zeros(len(epochs))
            mean_val = np.zeros(len(epochs))

        # Update mean values
        mean_train += train_values
        mean_val += val_values

        # Plot individual fold lines (lighter)
        plt.plot(epochs, train_values, 'b-', alpha=0.2, label=f'Train Fold {fold}' if fold == 1 else "")
        plt.plot(epochs, val_values, 'r-', alpha=0.2, label=f'Val Fold {fold}' if fold == 1 else "")

    # Calculate and plot mean values
    if mean_train is not None:
        num_folds = len(metrics_dict)
        mean_train /= num_folds
        mean_val /= num_folds

        plt.plot(epochs, mean_train, 'b-', linewidth=2, label='Mean Train')
        plt.plot(epochs, mean_val, 'r-', linewidth=2, label='Mean Val')

        # Calculate the gap between train and val at the last epoch
        final_gap = mean_train[-1] - mean_val[-1]

        # Add text annotation about overfitting
        if final_gap > 0.05:  # Threshold for concerning gap
            overfitting_level = "High" if final_gap > 0.1 else "Moderate"
            plt.text(
                max_epochs * 0.7,
                0.3,
                f"Potential {overfitting_level} Overfitting\nFinal Gap: {final_gap:.4f}",
                fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8)
            )
        else:
            plt.text(
                max_epochs * 0.7,
                0.3,
                f"Low Overfitting Risk\nFinal Gap: {final_gap:.4f}",
                fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8)
            )

    # Add labels and title
    plt.xlabel('Epochs')
    plt.ylabel(f'{metric_name.capitalize()} Accuracy')
    plt.title(f'Training vs Validation {metric_name.capitalize()} Accuracy - {data_type.capitalize()} Data')
    plt.ylim(0, 1.05)
    plt.grid(True)
    plt.legend(loc='lower right')

    # Save figure
    plt.savefig(f'/content/{data_type}_overfitting_check.png', dpi=300, bbox_inches='tight')
    plt.close()

    return mean_train, mean_val

def plot_loss_curves(metrics_dict, data_type):
    """
    Plot training vs validation loss curves across all folds.

    Args:
        metrics_dict: Dictionary with metrics dataframes for each fold
        data_type: 'enhanced' or 'original'

    Returns:
        None, saves the plot to disk
    """
    plt.figure(figsize=(14, 10))

    # For storing mean values across folds
    mean_train_loss = None
    mean_val_loss = None
    max_epochs = 0

    # Plot each fold
    for fold, df in metrics_dict.items():
        if not 'train/loss' in df.columns or not 'val/loss' in df.columns:
            print(f"Warning: Loss metrics not found in fold {fold}")
            continue

        epochs = df['epoch'].values
        train_loss = df['train/loss'].values
        val_loss = df['val/loss'].values

        # Update max epochs
        max_epochs = max(max_epochs, len(epochs))

        # Initialize mean arrays if needed
        if mean_train_loss is None:
            mean_train_loss = np.zeros(len(epochs))
            mean_val_loss = np.zeros(len(epochs))

        # Update mean values
        mean_train_loss += train_loss
        mean_val_loss += val_loss

        # Plot individual fold lines (lighter)
        plt.plot(epochs, train_loss, 'b-', alpha=0.2, label=f'Train Loss Fold {fold}' if fold == 1 else "")
        plt.plot(epochs, val_loss, 'r-', alpha=0.2, label=f'Val Loss Fold {fold}' if fold == 1 else "")

    # Calculate and plot mean values
    if mean_train_loss is not None:
        num_folds = len(metrics_dict)
        mean_train_loss /= num_folds
        mean_val_loss /= num_folds

        plt.plot(epochs, mean_train_loss, 'b-', linewidth=2, label='Mean Train Loss')
        plt.plot(epochs, mean_val_loss, 'r-', linewidth=2, label='Mean Val Loss')

        # Calculate the gap between train and val loss at the last epoch
        final_gap = mean_val_loss[-1] - mean_train_loss[-1]

        # Add text annotation about overfitting
        if final_gap > 0.1:  # Threshold for concerning gap
            overfitting_level = "High" if final_gap > 0.2 else "Moderate"
            plt.text(
                max_epochs * 0.7,
                mean_val_loss.max() * 0.7,
                f"Potential {overfitting_level} Overfitting\nFinal Loss Gap: {final_gap:.4f}",
                fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8)
            )
        else:
            plt.text(
                max_epochs * 0.7,
                mean_val_loss.max() * 0.7,
                f"Low Overfitting Risk\nFinal Loss Gap: {final_gap:.4f}",
                fontsize=12,
                bbox=dict(facecolor='white', alpha=0.8)
            )

    # Add labels and title
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training vs Validation Loss - {data_type.capitalize()} Data')
    plt.grid(True)
    plt.legend(loc='upper right')

    # Save figure
    plt.savefig(f'/content/{data_type}_loss_curves.png', dpi=300, bbox_inches='tight')
    plt.close()

    return mean_train_loss, mean_val_loss

def plot_combined_metrics(enhanced_metrics, original_metrics):
    """
    Plot a combined comparison of enhanced vs original data metrics

    Args:
        enhanced_metrics: Dict with metrics for enhanced data
        original_metrics: Dict with metrics for original data

    Returns:
        None, saves the plot to disk
    """
    # Extract mean accuracy curves
    e_train_acc, e_val_acc = plot_training_validation_curves(enhanced_metrics, "enhanced", "top1")
    o_train_acc, o_val_acc = plot_training_validation_curves(original_metrics, "original", "top1")

    if e_train_acc is None or o_train_acc is None:
        print("Warning: Cannot create combined plot due to missing data")
        return

    # Create combined plot
    plt.figure(figsize=(14, 10))

    # Create x-axis for epochs
    epochs = range(1, len(e_train_acc) + 1)

    # Plot all curves
    plt.plot(epochs, e_train_acc, 'b-', linewidth=2, label='Enhanced - Train Acc')
    plt.plot(epochs, e_val_acc, 'b--', linewidth=2, label='Enhanced - Val Acc')
    plt.plot(epochs, o_train_acc, 'g-', linewidth=2, label='Original - Train Acc')
    plt.plot(epochs, o_val_acc, 'g--', linewidth=2, label='Original - Val Acc')

    # Calculate overfitting gaps
    enhanced_gap = e_train_acc[-1] - e_val_acc[-1]
    original_gap = o_train_acc[-1] - o_val_acc[-1]

    # Add text annotations about overfitting comparison
    plt.text(
        len(epochs) * 0.6,
        0.4,
        f"Enhanced Data - Train/Val Gap: {enhanced_gap:.4f}\nOriginal Data - Train/Val Gap: {original_gap:.4f}",
        fontsize=12,
        bbox=dict(facecolor='white', alpha=0.8)
    )

    # Add labels and title
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Comparing Overfitting: Enhanced vs Original Resolution')
    plt.ylim(0, 1.05)
    plt.grid(True)
    plt.legend(loc='lower right')

    # Save figure
    plt.savefig('/content/resolution_comparison_overfitting.png', dpi=300, bbox_inches='tight')
    plt.close()

def main():
    # Extract metrics
    enhanced_metrics = extract_metrics_from_csv('enhanced')
    original_metrics = extract_metrics_from_csv('original')

    # Check if we found metrics
    if len(enhanced_metrics) == 0 and len(original_metrics) == 0:
        print("No metrics found. Make sure training has completed and results.csv files exist.")
        return

    # Plot accuracy curves
    if len(enhanced_metrics) > 0:
        print("Plotting enhanced data metrics...")
        plot_training_validation_curves(enhanced_metrics, 'enhanced')
        plot_loss_curves(enhanced_metrics, 'enhanced')

    if len(original_metrics) > 0:
        print("Plotting original data metrics...")
        plot_training_validation_curves(original_metrics, 'original')
        plot_loss_curves(original_metrics, 'original')

    # If we have both types, create combined comparison
    if len(enhanced_metrics) > 0 and len(original_metrics) > 0:
        print("Creating combined comparison plot...")
        plot_combined_metrics(enhanced_metrics, original_metrics)

    print("\nPlotting complete! Check the following files for visualizations:")
    if len(enhanced_metrics) > 0:
        print("- /content/enhanced_overfitting_check.png")
        print("- /content/enhanced_loss_curves.png")
    if len(original_metrics) > 0:
        print("- /content/original_overfitting_check.png")
        print("- /content/original_loss_curves.png")
    if len(enhanced_metrics) > 0 and len(original_metrics) > 0:
        print("- /content/resolution_comparison_overfitting.png")

if __name__ == "__main__":
    main()

Plotting enhanced data metrics...


<ipython-input-12-c248a87185b8>:123: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(loc='lower right')


Plotting original data metrics...
Creating combined comparison plot...

Plotting complete! Check the following files for visualizations:
- /content/enhanced_overfitting_check.png
- /content/enhanced_loss_curves.png
- /content/original_overfitting_check.png
- /content/original_loss_curves.png
- /content/resolution_comparison_overfitting.png
